In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path

import torch
import wandb

from config import Environment, TrainConfig
from denoising.train import prepare_training
from denoising.utils import seed_everything

In [4]:
CWD = Path.cwd()
env = Environment(_env_file=CWD / '../env')
wandb.login(key=env.wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/d.nesterov/.netrc
wandb: Currently logged in as: dmitrylala to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Конфиг обучения и подготовка

In [5]:
run_idx = 33
save_dir = CWD / 'mri'


cfg = TrainConfig(
    # Datasets params
    train_dset='mri_gt_train',
    test_dset='mri_gt_val',
    train_batch_size=64,
    test_batch_size=128,
    # Model params
    name_model='mri-hno-gt-optuned-v2',
    cfg_fno={
        'n_modes': (16, 16),
        'in_channels': 1,
        'hidden_channels': 32,
        'lifting_channel_ratio': 16,
        'projection_channel_ratio': 2,
        'out_channels': 1,
        'factorization': 'dense',
        'n_layers': 4,
        'rank': 0.42,
        'spectral': 'hartley',
    },
    # Run params
    random_seed=42,
    device='cuda:2',
    run_name=f'Run {run_idx}, HNO gt optuned v2',
    save_weights_path=save_dir / f'run-{run_idx}-weights.pt',
    # Train params
    n_epochs=90,
    lr=5e-3,
    wandb_log=True,
    save_dir=save_dir / f'run-{run_idx}',
    verbose=True,
)

In [6]:
trainer, train_kwargs, run = prepare_training(env, cfg)

Got n_samples = 8380  in dataset mri_pm_train        with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_pm_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 6704  in dataset mri_gt_train        with sample size = torch.Size([1, 145, 145])
Got n_samples = 1676  in dataset mri_gt_val          with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_gt_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 137   in dataset bsd_synth_0.01_train with sample size = torch.Size([1, 321, 481])
Got n_samples = 77    in dataset bsd_synth_0.01_test with sample size = torch.Size([1, 321, 481])
Got n_samples = 12296 in dataset sidd_train          with sample size = torch.Size([3, 512, 512])
Got n_samples = 3008  in dataset sidd_test           with sample size = torch.Size([3, 512, 512])
torch.Size([64, 1, 145, 145]) torch.Size([64, 1, 145, 145])
Loaded  model mri-fno-neuralop with n_parameters = 201044

Logging to wandb enabled: True


# Запуск

In [7]:
seed_everything(cfg.random_seed)
trainer.train(**train_kwargs)

Training on 6704 samples
Testing on [1676] samples         on resolutions ['test'].
Raw outputs of shape torch.Size([64, 1, 145, 145])
[0] time=43.49, avg_loss=0.3341, train_err=21.3285
Eval: test_h1=0.3015, test_l2=0.0976
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-33
[1] time=39.35, avg_loss=0.2990, train_err=19.0893
Eval: test_h1=0.2922, test_l2=0.0933
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-33
[2] time=39.66, avg_loss=0.2904, train_err=18.5399
Eval: test_h1=0.2849, test_l2=0.0878
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-33
[3] time=40.00, avg_loss=0.2817, train_err=17.9860
Eval: test_h1=0.2767, test_l2=0.0837
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-33
[4] time=40.75, avg_loss=0.2746, train_err=17.5353
Eval: test_h1=0.2704, test_l2=0.0834
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-33
[5] time=39.34, avg_loss=0.2683, train_err=17

{'train_err': 11.776624761308943,
 'avg_loss': 0.1844489260049879,
 'epoch_train_time': 39.887530183419585,
 'test_h1': tensor(0.1885, device='cuda:2'),
 'test_l2': tensor(0.0558, device='cuda:2'),
 'test_outputs': <wandb.sdk.data_types.image.Image at 0x7fb4bb677c40>}

In [8]:
if run is not None:
    run.finish()

avg_loss,█▇▇▆▆▆▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
lr,██▇▇▆▅▅▃▂▂▁▁▁▁▂▂▃▃▅▅▇████████▇▅▅▅▄▃▂▂▁▁▁
test_h1,█▆▆▆▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
test_l2,█▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
time,▃▃▂▂▂▂▂▃▂▂▂▁▂▂▁▂▂▄▄▇▇▂▃▅█▅▃▃▃▄▅▆█▄▃▃█▆▄▃
train_err,█▆▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
avg_loss,0.18445
lr,0
test_h1,0.18847
test_l2,0.0558
time,39.88753


In [9]:
torch.save(trainer.model.to('cpu').state_dict(), cfg.save_weights_path)
print(f'Saved to {cfg.save_weights_path}')

Saved to /home/d.nesterov/denoising-fno/notebooks/mri/run-33-weights.pt
